# Homework 1: Locally Sensitive Hashing
Locality Sensitive Hashing (LSH) is a technique used in computer science to solve the approximate or exact Near Neighbor Search in high-dimensional spaces. It is used to find similar items in a large dataset by hashing input items so that similar items map to the same "buckets" with high probability. LSH is commonly used in recommendation systems, image and audio recognition, and data mining.

In this particular notebook we will implement a simplified version of the LSH algorithm for to compare texts and find how similar are. We will implement 4 classes which will help us to compute how similar 2 texts are. Those classes are: Shingling, CompareSets, MinHasing and CompareSignatures.

## Dataset
As a dataset, we have used the following texts:
 - Lorem Ipsum with 5 paragraphs (https://www.lipsum.com/)\[1.txt\]
 - Lorem Ipsum with 7 paragraphs (https://www.lipsum.com/)\[2.txt\]
 - Quijote de la Mancha by Miguel de Cervantes (https://www.gutenberg.org/cache/epub/60884/pg60884.txt)\[3.txt\]
 - The Adventures of Sherlock Holmes by Arthur Conan Doyle (https://www.gutenberg.org/cache/epub/1661/pg1661.txt)\[4.txt\]
 - The picture of Dorian Gray by Oscar Wilde (https://www.gutenberg.org/cache/epub/174/pg174.txt)\[5.txt\]
 - Beyond good and evil by Friedrich Wilheim Nietzsche (https://www.gutenberg.org/cache/epub/4363/pg4363.txt)\[6.txt\]

This dataset has been selected like this, so it has two text which should be fairly similar (1 & 2), a text which should be fairly different (3) and three texts which should have something in common even though they may be different (4, 5 & 6).

In this notebook we have implemented 4 classes which we will help us to compute the similarity of 2 texts. Those classes are: Shingling, CompareSets, MinHashing and CompareSignatures.

In [11]:
import os

# define the path to the dataset folder
dataset_path: str = "dataset"

# create an empty list to store the texts
texts: list[str] = []

# loop through the files in the dataset folder
for filename in os.listdir(dataset_path):
    # check if the file is a text file
    if filename.endswith(".txt"):
        # open the file and read its contents
        with open(os.path.join(dataset_path, filename), "r") as f:
            text = f.read()
            texts.append(text)

# print the texts
for i, text in enumerate(texts):
    print(f"Text {i+1}:")
    print(text)
    print("-"*132)


Text 1:
HAPTER I.


The studio was filled with the rich odour of roses, and when the light
summer wind stirred amidst the trees of the garden, there came through
the open door the heavy scent of the lilac, or the more delicate
perfume of the pink-flowering thorn.

From the corner of the divan of Persian saddle-bags on which he was
lying, smoking, as was his custom, innumerable cigarettes, Lord Henry
Wotton could just catch the gleam of the honey-sweet and honey-coloured
blossoms of a laburnum, whose tremulous branches seemed hardly able to
bear the burden of a beauty so flamelike as theirs; and now and then
the fantastic shadows of birds in flight flitted across the long
tussore-silk curtains that were stretched in front of the huge window,
producing a kind of momentary Japanese effect, and making him think of
those pallid, jade-faced painters of Tokyo who, through the medium of
an art that is necessarily immobile, seek to convey the sense of
swiftness and motion. The sullen murmur of 

## Shingling
Shingling is a technique used in text analysis to represent a document as a set of overlapping subsequences of fixed length k, called k-shingles. To compute shingles, we slide a window of size k over the document and extract the k-length substrings that fall within the window. We then store these substrings as a set, which represents the shingles of the document.

In [12]:
class Shingling():
    
    def __init__(self, k: int):
        self.k: int = k
        
    def get_shingles(self, document: str) -> set:
        """
        This method constructs k-shingles of a given length k from a given document.
        """
        shingles: set = set()
        for i in range(len(document) - self.k + 1):
            shingle: str = document[i:i+self.k]
            shingles.add(shingle)
        return shingles
    
    def get_hashed_shingles(self, document: str):
        """
        This method computes a hash value for each unique shingle and represents the document in the form of an ordered set of its hashed k-shingles.
        """
        shingles: set = self.get_shingles(document)
        hashed_shingles: list[int] = [hash(shingle) for shingle in shingles]
        hashed_shingles.sort()
        return hashed_shingles

As an example to show what is shingling, we will use the following text:
> "The quick brown fox jumps over the lazy dog"

In [13]:
text = "The quick brown fox jumps over the lazy dog"
shingling: Shingling = Shingling(k=5)
shingles: list[str] = shingling.get_shingles(text)
print(shingles)
shingles_hash: list[int] = shingling.get_hashed_shingles(text)
print("-"*132)
print(shingles_hash)



{' the ', 'ps ov', 'own f', ' jump', 'y dog', 'uick ', 'x jum', ' brow', 'rown ', 'azy d', 'n fox', 'ox ju', 'wn fo', 's ove', 'er th', ' quic', 'brown', 'over ', 'he la', 'lazy ', 'the l', ' fox ', 'ick b', 'jumps', 'k bro', 'mps o', 'The q', ' lazy', 'ver t', 'e laz', 'zy do', 'he qu', 'r the', 'quick', 'umps ', 'e qui', 'ck br', 'fox j', ' over'}
------------------------------------------------------------------------------------------------------------------------------------
[-9183468719421131492, -9160573196651070498, -8686590549064579419, -8576912921588956908, -8486209293372859830, -8035514751081473557, -7301381534162091266, -6942666486512717320, -6870951987087571257, -6125146351140126477, -6059456853732382328, -5623278449309964217, -5110574680244950947, -5038623011202038515, -4884758002727058576, -4681205661868904004, -3603449078763937169, -2433448449045537089, -2102532412785618259, -1348799055194476030, -1004775896417412909, -911614797966191206, -792037816140207270, -692487238

Now we do it with our text. We use a k=9 as we are analyzing text from books instead of emails. In the following lines we are doing exactly the same as we showed in the example above.

In [14]:
shingling: Shingling = Shingling(k=9)

hashed_shingles_list: list[int] = []

for text in texts:
    hashed_shingles: int = shingling.get_hashed_shingles(text)
    hashed_shingles_list.append(hashed_shingles)
    
for i, hashed_shingles in enumerate(hashed_shingles_list):
    print(f"Hashed shingles for Text {i+1}:")
    print(hashed_shingles)
    print("-"*132)


Hashed shingles for Text 1:
[-9211198511936942536, -9207289093226432091, -9202756026031104848, -9189841881721657399, -9184723919433772362, -9164750270807723127, -9163345424511707644, -9163156426391938491, -9154969343552991580, -9151975312355243358, -9145952001961894050, -9142649405963196318, -9122820578701755313, -9119853111418622318, -9119194925860318471, -9114476631502841516, -9108618811510550817, -9104971378653136144, -9093636160742358732, -9087611301845274766, -9086677924538704585, -9077605970134746040, -9074010212022360185, -9070239636139681601, -9068632988197667256, -9058805765128030375, -9056441211902613318, -9051602137561169041, -9049015761532032534, -9046090683186317717, -9038197187167348659, -9033316382018915956, -9027199535321574767, -9024911001594753231, -9022888852969732891, -9021068168794248313, -9007308505369488830, -9002853000166018090, -9000905310754552718, -8999970657774859197, -8989760198387034895, -8986646331017379094, -8982708872764084699, -8970526852118845748, -89

## CompareSets


In [15]:
class CompareSets():
    
    def __init__(self):
        pass
    
    def jaccard_similarity(self, set1: set, set2: set) -> float:
        """
        This method computes the Jaccard similarity of two sets.
        """
        intersection_size: int = len(set1.intersection(set2))
        union_size: int = len(set1.union(set2))
        jaccard_similarity: float = intersection_size / union_size
        return jaccard_similarity

In [16]:
compare_sets: CompareSets = CompareSets()

# compute the Jaccard similarity of the hashed shingles for Text 1 and Text 2
jaccard_similarity: float = compare_sets.jaccard_similarity(set(hashed_shingles_list[0]), set(hashed_shingles_list[1]))

print(f"The Jaccard similarity of Text 1 and Text 2 is {jaccard_similarity:.2f}")

The Jaccard similarity of Text 1 and Text 2 is 0.01


## MinHashing

## CompareSignatures


## LSH